#  [Local Binary Convolutional Neural Networks](https://openaccess.thecvf.com/content_cvpr_2017/papers/Juefei-Xu_Local_Binary_Convolutional_CVPR_2017_paper.pdf)  
# Not ready yet

The image is converted into a gray sacale one.  
for each pixel we calculate if the neighbor one is greater than it.  
  
![Convolution Diagram](../../assets/lbp_thresholding.jpg)  
The eigth binaries are concatenated to represent a 256 base integer.  

![Convolution Diagram](../../assets/lbp_calculation-1024x299.jpg)

![Convolution Diagram](../../assets/lbp_to_output-1024x519.jpg)


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from tensorflow.keras import backend as K
from matplotlib import pyplot
from PIL import Image

In [2]:

class LBPLayer(layers.Layer):
    def __init__(self):
        super(LBPLayer, self).__init__()

    def build(self, input_shape):
        self.lbp_kernel = [1 << 7, 1 << 6, 1 << 5, 1 << 4, 1 << 3, 1 << 2, 1 << 1, 1 << 0]  
        self.H = [[ 
          [[ 0, 0, 0, 0, 0, 0, 0, 1]],
          [[ 1, 0, 0, 0, 0, 0, 0, 0]],
          [[ 0, 1, 0, 0, 0, 0, 0, 0]]
          ],
          [ 
          [[ 0, 0, 0, 0, 0, 0, 1, 0]],
          [[-1,-1,-1,-1,-1,-1,-1,-1]],
          [[ 0, 0, 1, 0, 0, 0, 0, 0]]
          ],
          [ 
          [[ 0, 0, 0, 0, 0, 1, 0, 0]],
          [[ 0, 0, 0, 0, 1, 0, 0, 0]],
          [[ 0, 0, 0, 1, 0, 0, 0, 0]]
        ]]

        self.kernel = tf.constant(self.H , dtype=tf.float32)

    def call(self, image_original):
        image = image_original #  tf.image.rgb_to_grayscale(image_original)
        image = tf.reshape(image, [-1, height_reshaped, width_reshaped, 1])
        conv = tf.nn.conv2d(image, self.kernel, strides=[1, 1, 1, 1], padding='SAME')
        sign = tf.math.sign(tf.nn.relu(conv))
        result = tf.reduce_sum(tf.multiply(sign,  self.lbp_kernel), 3)
        return conv
        

In [3]:
height_reshaped = 64
width_reshaped = 64
inputs = keras.Input(shape=(height_reshaped, width_reshaped, 3))
gray_layer = tf.image.rgb_to_grayscale(inputs)
outputs = LBPLayer()(gray_layer)

model = keras.Model(inputs, outputs)

In [4]:
im = cv2.imread('../../assets/Lena.png')
im = cv2.resize(im,(int(width_reshaped),int(height_reshaped)))
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
print(gray)


[[161 159 158 ... 126 118 164]
 [157 156 156 ... 129 103  47]
 [157 157 156 ...  95  51  47]
 ...
 [ 66 102 176 ... 102  65  61]
 [ 59  72 189 ...  84  58  52]
 [ 51  56 198 ...  52  54  93]]


In [5]:
output = model.predict(np.array([im]))[0]
output = np.array(output)
output = output.astype(int)
print(output)

pyplot.imshow(output, cmap = 'gray') 
pyplot.show()

[[[-141 -141   -3 ... -141 -141 -141]
  [-138 -138   -1 ...   -3    3 -138]
  [-136 -136   -2 ...   -3    1 -136]
  ...
  [-107 -107   -7 ...   -2    0 -107]
  [ -99  -99   44 ...    9    7  -99]
  [-144 -144 -144 ...  -57  -44 -144]]

 [[   6    3   -1 ... -135 -135 -135]
  [   4    3    0 ...    1    1    8]
  [   2    0   -2 ...    0    0    3]
  ...
  [  -1   -9  -22 ...    4   -4   -1]
  [  13   57  -45 ...   -7   22   20]
  [ 102  -41  -41 ...    5   45   58]]

 [[   0   -1   -1 ... -135 -135 -135]
  [   0    0    0 ...    1    1    0]
  [   0   -2    0 ...    1    0    0]
  ...
  [  30    7  -32 ...   -4   34   25]
  [  39   -5   -5 ...   -1   32   62]
  [   0  -41  -41 ...   -1    5   45]]

 ...

 [[  22   59   37 ...  -58  -58  -58]
  [  22   77   74 ...  -42  -37  -14]
  [   3  -52  -43 ... -102  -74  -51]
  ...
  [   4  -20  -37 ...   15    9  -14]
  [  16    3   -3 ...   18   37   41]
  [   6  -53  -53 ...   -3    3   20]]

 [[   5   42   13 ...  -53  -53  -53]
  [  28  102

TypeError: Invalid shape (64, 64, 8) for image data

In [24]:
height_reshaped = 3
width_reshaped = 3
inputs = keras.Input(shape=(height_reshaped, width_reshaped, 1))
outputs = LBPLayer()(inputs)
model = keras.Model(inputs, outputs)
im = [[5, 8, 1],
      [5, 4, 1],
      [3, 7, 2]]
output = model.predict(np.array([im]))[0]
output = np.array(output)
output = output.astype(int)

print(output)
print(output.shape)
pyplot.imshow(output, cmap = 'gray') 
pyplot.show()



[[[-5 -5  3 -1  0 -5 -5 -5]
  [-8 -8 -7 -7 -4 -3 -3 -8]
  [-1 -1 -1 -1  0  3  7 -1]]

 [[ 0  3 -1  2 -2 -5 -5 -5]
  [ 4 -3 -3 -2  3 -1  1  1]
  [ 0 -1 -1 -1  1  6  3  7]]

 [[ 2  1  4 -3 -3 -3 -3 -3]
  [-3 -6 -5 -7 -7 -7 -4 -2]
  [-1 -2 -2 -2 -2 -2  5  2]]]
(3, 3, 8)


TypeError: Invalid shape (3, 3, 8) for image data